In [1]:
from matplotlib import pylab
import nengo
import random
import numpy as np
import gzip as gz
import cPickle
from cPickle import load
try:
    import Image
except ImportError:
    from PIL import Image
from scipy.sparse.linalg import svds
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#%matplotlib inline #Makes visualizations appar inline (Commented out because animation popup as new window)

In [2]:
def load_img():
    a = Image.open("three64.png") #three64, three128, three256, three512 http://www.flaticon.com/free-icon/number-three-in-a-circle_56587
    a = np.array(a)
    
    #32*32 img came with shape (32,32,4)
    #a = a[:,:,:-3]
    #a = a.squeeze()
    
    return a.reshape(dim**2,)

In [3]:
def rotate_img(img, degrees):
    '''Rotates image the degrees passed in counterclockwise
    Reshapes image to original shape
    '''
    original = img.shape
    
    newImg = scipy.ndimage.interpolation.rotate(np.reshape(img, (dim,dim), 'F'),degrees,reshape=False)
    newImg = np.reshape(newImg, original, 'F')
    return newImg

In [4]:
conn_synapse = 0.1 #post synaptic time constant to use for filtering (pstc) - what does changing this do?
probe_synapse = 0.01 #pstc
multiplier = 2 #not used
n_neurons = 5000
direct = True #Direct - function computed explicitly instead of in neurons 
stop_time = 3.0
run_time = 3.0 #in seconds

In [5]:
dim = 64 #size of the image
img = load_img()

In [6]:
def stim_func(t):
    '''returns the image for first 0.1s'''
    if t < 0.1:
        return img
    else:
        return [0 for _ in range(len(img))]

In [7]:
def node_func(t,x):
        return rotate_img(x,1)

In [8]:
with nengo.Network() as net:
    
    if direct:
        neuron_type = nengo.Direct() #function computed explicitly, instead of in neurons
    else:
        neuron_type = nengo.LIF() #spiking version of the leaky integrate-and-fire neuron model

    #Input stimulus - provide data to the ensemble
    ipt = nengo.Node(stim_func)
    
    ensArr = nengo.networks.EnsembleArray(100, dim**2, ens_dimensions=1) #incresing num neurons has small effect on run time
    
    node = nengo.Node(node_func,size_in = dim**2, size_out =dim**2)
    
    for i in range(ensArr.n_ensembles):
        nengo.Connection(ipt[i],ensArr.ea_ensembles[i])
        nengo.Connection(ensArr.ea_ensembles[i],node[i])
        nengo.Connection(node[i],ensArr.ea_ensembles[i])
        
    ens = nengo.Ensemble(n_neurons, dimensions = dim**2)
    nengo.Connection(node, ens)
    
    probe = nengo.Probe(ens, attr='decoded_output',#sample_every=0.001,
                       synapse=probe_synapse)
        


   

In [9]:
sim = nengo.Simulator(net)

In [10]:
sim.run(run_time)

Simulation finished in 0:04:03.                                                 


KeyboardInterrupt: 

In [ ]:
pylab.imshow(np.reshape(img, (dim,dim), 'F'), cmap='Greys_r')

In [ ]:
'''Image at stop time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][int(stop_time*1000)-1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [ ]:
'''Image at start time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [ ]:
'''Animation for Probe output'''
fig = plt.figure()

def updatefig(i):
    im = pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][i]],
                                 (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)
    
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=1, blit=True)
plt.show()


In [ ]:
# save the output
#cPickle.dump(sim.data[probe], open( "Buffer_rotations_larger_images_64.p", "wb" ) )